<a href="https://colab.research.google.com/github/ilitteri/7506-Datos/blob/main/tp1_parte1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from datetime import datetime, time
import random
import json

In [6]:
def path_for(dataset: str) -> str:
    return f'/content/drive/MyDrive/Dataset/{dataset}.csv'

def nan_count_in_series(column: pd.Series) -> int:
    return column.isna().sum()

In [7]:
businesses = pd.read_csv(path_for("business"))
review = pd.read_csv(path_for("review"))

# P19

Fernando quiere festejar su cumpleaños, que cae jueves, al salir de la oficina a las 19 hs. Sus amigos viven distribuidos en la ciudad de Philadelphia, por lo que es indiferente la ubicación del lugar, siempre y cuando esté en la ciudad. Quiere que le ofrezcamos el top 3 de lugares de comida italiana, generando el ranking según la cantidad de reviews de 5 estrellas que hayan recibido. Devolver algunas referencias (al azar) de cada uno de ellos, para ayudarlo a decidir en cual realizar la reserva.

In [11]:
def trasnocha(hora_de_apertura, hora_de_cierre):
    return hora_de_apertura > hora_de_cierre

def hora_esta_entre(hora_de_apertura: datetime.time, hora_de_cierre: datetime.time, hora_a_verificar: datetime.time):
    return hora_de_apertura <= hora_a_verificar < hora_de_cierre if not trasnocha(hora_de_apertura, hora_de_cierre) else hora_a_verificar >= hora_de_apertura or hora_a_verificar <= hora_de_cierre

def transformar_a_tiempo(a_time: str):
    return time(*list(map(int, a_time.split(':'))))

def esta_abierto_a_las_19(rango: str):
    hora_de_apertura, hora_de_cierre = rango.split('-')
    return hora_esta_entre(transformar_a_tiempo(hora_de_apertura), transformar_a_tiempo(hora_de_cierre), time(19, 0))

def mezclar_reviews(id_de_lugar_en_top, df):
    recomendaciones = df.loc[df['business_id'] == id_de_lugar_en_top]['text'].tolist()
    random.shuffle(recomendaciones)
    return recomendaciones

lugares_accesibles_para_fernando = businesses[(businesses['city'] == 'Philadelphia') & businesses['hours'].notnull() & businesses['hours'].str.contains('Thursday') & businesses['categories'].str.contains('Italian')]
lugares_accesibles_para_fernando['hours'] = lugares_accesibles_para_fernando['hours'].map(lambda horario: esta_abierto_a_las_19(json.loads(horario.replace("'", '"'))['Thursday']))
lugares_accesibles_para_fernando[lugares_accesibles_para_fernando['hours'] == True]

foo = review[review['stars'] == 5.0].merge(lugares_accesibles_para_fernando, left_on='business_id', right_on='business_id')

ids_de_top_3_lugares_de_comida_italiana_en_philadelphia = foo.groupby('business_id')['review_id'].count().sort_values(ascending=False)[:3].index

top_3_lugares_de_comida_italiana_en_philadelphia = ids_de_top_3_lugares_de_comida_italiana_en_philadelphia.map(lambda business_id: businesses[businesses['business_id'] == business_id]['name'].values[0]).to_list()

reviews_random_por_lugar_en_top = [mezclar_reviews(id_de_lugar_en_top, foo) for id_de_lugar_en_top in ids_de_top_3_lugares_de_comida_italiana_en_philadelphia]

print('TOP 3 LUGARES DE COMIDA ITALIANA EN PHILADELPHIA PARA RECOMENDAR A FERNANDO')
print('===========================================================================')
print()
for i, reviews_de_lugar in enumerate(reviews_random_por_lugar_en_top):
    print(f'({i+1}°) {top_3_lugares_de_comida_italiana_en_philadelphia[i]}')
    print()
    print(f'Review 1: {reviews_de_lugar[0]}')
    print(f'Review 2: {reviews_de_lugar[1]}')
    print(f'Review 3: {reviews_de_lugar[2]}')
    print()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


TOP 3 LUGARES DE COMIDA ITALIANA EN PHILADELPHIA PARA RECOMENDAR A FERNANDO

(1°) Barbuzzo

Review 1: SO GOOD. One of my favorite restaurants in philly so far. The appetizers, mains, and desserts were all amazing.

Appetizers:
1. Charcuterie board - delicious and highly recommend to share with a group; not as big of a fan of the bread that came with it
2. Whipped ricotta - the ricotta and bread were both so good (the bread was soft but toasted so crunch in all the right parts!!)

Uovo pizza is a YES.

Desserts - by then I was too full, but my friend definitely had foodgasm with the tiramisu. I preferred the caramel dessert, but would recommend both!
Review 2: I had one night in Philly and I choose Barbuzzo!
The Gayborhood is just full of amazing restaurants and Barbuzza just adds another to the list.  I sat at the bar and enjoyed the row home feel of the restaurant by looking down the long galley.  I started with the smoked trout tartine which was phenomenal. The basil and dill just pa